In [1]:
import cvxpy as cp
import numpy as np

In [65]:
# Define some basic matrices
sigma_x = np.matrix([[0,1],[1,0]])
sigma_y = np.matrix([[0,1j],[-1j,0]])
sigma_z = np.matrix([[1,0],[0,-1]])
sigma_0 = np.matrix([[1,0],[0,1]])
identity = np.matrix([[1,0],[0,1]])

pauli_list = [sigma_0, sigma_x, sigma_y, sigma_z]
logical_basis_list = [np.kron(np.kron(s1, s2), s3) for s1 in pauli_list for s2 in pauli_list for s3 in pauli_list]

Bs_0 = np.matrix([[1],[0]])
Bs_1 = np.matrix([[0],[1]])
Ls_0 = np.kron(np.kron(Bs_0, Bs_0), Bs_0)
Ls_1 = np.kron(np.kron(Bs_1, Bs_1), Bs_1)



rho_vector = 0.5*(np.kron(Ls_0, Ls_0) + np.kron(Ls_1, Ls_1))
print(rho_vector.shape)
print(len(logical_basis_list))


(64, 1)
64


In [66]:
# Define errors
p = 0.1
E0 = np.sqrt(1-3*p)*np.kron(np.kron(identity,identity), identity)
E1 = np.sqrt(p)*np.kron(np.kron(sigma_x,identity), identity)
E2 = np.sqrt(p)*np.kron(np.kron(identity,sigma_x), identity)
E3 = np.sqrt(p)*np.kron(np.kron(identity,identity), sigma_x)
Error_list = [E0, E1, E2, E3]

C_epsilon = sum([np.kron(Ei, np.matrix(np.identity(2**3)))*rho_vector*np.transpose(rho_vector)*np.kron(Ei, np.matrix(np.identity(2**3))) for Ei in Error_list])
# Logic state

(64, 64)

In [71]:
# Set the SDP parameters    
C = np.matrix(np.identity(64))/8
A = []
b = []
for i in range(64):
    for j in range(1,64):
        A.append(np.kron(logical_basis_list[i], logical_basis_list[j]))
        b.append(-np.trace(np.kron(logical_basis_list[i], logical_basis_list[j])*C_epsilon))
        
p = len(b)  
n = 64
print(p)

4032


In [72]:
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((64,64), complex=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i]@X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(C@X))),
                  constraints)
prob.solve()

0.9375017975804946

In [75]:
# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 0.9375017975804946
A solution X is
[[ 1.74996723e-01+0.j -2.17369476e-10+0.j  5.18788999e-09+0.j ...
   2.98073310e-09+0.j  9.73041820e-10+0.j -1.74999795e-01+0.j]
 [-2.17369476e-10+0.j  3.49996290e-01+0.j -2.13689253e-08+0.j ...
  -1.90481499e-08+0.j -2.14544056e-10+0.j -5.40137504e-10+0.j]
 [ 5.18788999e-09+0.j -2.13689253e-08+0.j  3.49996289e-01+0.j ...
  -2.99779847e-07+0.j -9.28831175e-08+0.j -3.42885678e-09+0.j]
 ...
 [ 2.98073310e-09+0.j -1.90481499e-08+0.j -2.99779847e-07+0.j ...
   3.49996055e-01+0.j -6.30056185e-08+0.j -2.90340446e-09+0.j]
 [ 9.73041820e-10+0.j -2.14544273e-10+0.j -9.28831175e-08+0.j ...
  -6.30056185e-08+0.j  3.49996055e-01+0.j -7.81444131e-10+0.j]
 [-1.74999795e-01+0.j -5.40137504e-10+0.j -3.42885678e-09+0.j ...
  -2.90340446e-09+0.j -7.81444131e-10+0.j  1.74996496e-01+0.j]]


In [68]:
i

63